# THE BASICS

## Tensor Orders


1. $1^{st}$-Order Tensors - Vectors

    A $1^{st}$-Order Tensor is either a row or column vector. We often see it denoted as having shape of either $\mathbb{R}^{Mx1}$ or $\mathbb{R}^{1xN}$. Here, M dictates the number of rows (elements) in the column vector. N dictates the number of columns (elements) in the row vector. $\mathbb{R}$ represents the overall state of the $1^{st}$-Order tensor. It means that the tensor is constructed of all real values and is in the shape of {M x 1} or {1 x N}.
    
    $$
        A = 
        \begin{bmatrix}
        1 & 2 & 3
        \end{bmatrix}
    $$

    Where **A** is a 1 x 3 Vector

    $$
        B = 
        \begin{bmatrix}
        4 \\ 5 \\ 6
        \end{bmatrix}
    $$

    Where **B** is a 3 x 1 matrix.


2. $2^{nd}$-Order Tensors - Matrices

    A $2^{nd}$-Order Tensor is a matrix. Throughout the tutorial, we will only deal with real numbers. Therefore if **A** is a $2^{nd}$-Order Tensor having **M** rows and **N** columns, then A $\in \mathbb{R}^{MxN}$.

    $$
      A = 
      \begin{bmatrix}
      1 & 2 & 3 \\
      4 & 5 & 6 \\
      7 & 8 & 9
      \end{bmatrix}
    $$

    $ A \in \mathbb{R}^{3x3} $

3. $3^{rd}$-Order Tensors - Cubes

    A $3^{rd}$-Order Tensors can be thought of as a cube or stacked matrices. Therefore if **A** is a $3^{rd}$-Order Tensor having K instances of matrices of shape I rows and J columns, it would be denoted as **A** $\in \mathbb{R}^{IxJxK}$

    $$
        A = 
        \begin{bmatrix}
            \begin{bmatrix}
                1 & 2 & 3 \\
                4 & 5 & 6 \\
                7 & 8 & 9 \\
                10 & 11 & 12
            \end{bmatrix}
                
            \begin{bmatrix}
                11 & 12 & 13 \\
                14 & 15 & 16 \\
                17 & 18 & 19 \\
                20 & 21 & 22
            \end{bmatrix}

            \begin{bmatrix}
                31 & 32 & 33 \\
                34 & 35 & 36 \\
                37 & 38 & 39 \\
                40 & 41 & 42
            \end{bmatrix}
        \end{bmatrix}

        $ A \in \mathbb{R}^{4x3x3}

## Factorization


To understand the premise of this paper (this method) we first need to get an understanding and intrinsic level of intuition for factorization. 

### Example 1
Let's start with simple number factorization. If I give the number 24 and ask for the lowest factors possible, we can see that those factors are 3 and 2. This means that as long as I have both 3 and 2, I can reproduce 24. These are the most important building blocks of 24.

In [ ]:

from graphviz import Digraph
# Create Digraph object
dot = Digraph()

# Add nodes
dot.node('a', '24')
dot.node('b', '12')
dot.node('c', '2')
dot.node('d','6')
dot.node('e','3')

# Add edges
dot.edges(['ab', 'ac', 'bd', 'bc', 'de', 'dc'])

# Visualize the graph
dot

### Example 2

If **X** is defined as the set of numbers in:

$$
\{1,3,6,9,18,27,36,54,81,162\}
$$

We can actually store all the information we need about this set in the form of:

$$
\{1,3,6,9\}
$$

And this is becuase as long as we have these 4 numbers, we can create every value in **X**.


### Example 3


For a more relevant example. If the matrix **A** $\in \mathbb{R}^{4x4}$ is defined as:
$$
A = 
\begin{bmatrix}
2 & 12 & 6 & 18 \\
7 & 42 & 21 & 63 \\
4 & 24 & 12 & 36 \\
1 & 6 & 3 & 9
\end{bmatrix}
$$

We can actually find two vectors that encompass all the information in *A* but in a reduced form. Without formal proof, the two vectors:

$$
v1 = 
\begin{bmatrix}
2 \\ 7 \\ 4 \\ 1
\end{bmatrix}

, 

v2 = 
\begin{bmatrix}
1 \\ 6 \\ 3 \\ 9
\end{bmatrix}
$$

Will reproduce **A** when the outer product is taken between v1 and v2:

$$
v1 \otimes v2 = 

\begin{bmatrix}
2 \\ 7 \\ 4 \\ 1
\end{bmatrix}
\otimes
\begin{bmatrix}
1 & 6 & 3 & 9
\end{bmatrix}
=
\begin{bmatrix}
2*1 & 2*6 & 2*3 & 2*9 \\
7*1 & 7*6 & 7*3 & 7*9 \\
4*1 & 4*6 & 4*3 & 4*9 \\
1*1 & 1*6 & 1*3 & 1*9
\end{bmatrix}
=
\begin{bmatrix}
2 & 12 & 6 & 18 \\
7 & 42 & 21 & 63 \\
4 & 24 & 12 & 36 \\
1 & 6 & 3 & 9
\end{bmatrix}
$$


In [38]:
A

array([[ 2, 12,  6, 18],
       [ 7, 42, 21, 63],
       [ 4, 24, 12, 36],
       [ 1,  6,  3,  9]])

In [48]:
rank = 1
m,n = A.shape
U = np.random.rand(m,rank)
V = np.random.rand(rank,n)
print(f'U matrix shape: {U.shape}\nV matrix shape {V.shape}')

U matrix shape: (4, 1)
V matrix shape (1, 4)


In [75]:
def als(M, d, lam, iters):
    #intialize U
    #shape should be rank x M[rows]
    H = np.random.random((d, M.shape[0]))
    W = np.random.random((M.shape[1], d))
    
    #storage for errors per iteration
    train_errors = []

    

    
    #iterate
    for _ in range(iters):
        
        AW = M@H.T
        BW = H@H.T
        CW = W@BW
        W = W * (AW/CW)
        
        
        AH = M@W
        BH = W.T@W
        CH = H.T@BH.T
        H = H.T * (AH/CH)

    return H, W



In [84]:
h,w = als(A, 1, 0,100)

In [85]:
w@h.T

array([[ 3.71428571, 13.        ,  7.42857143,  1.85714286],
       [13.        , 45.5       , 26.        ,  6.5       ],
       [ 7.42857143, 26.        , 14.85714286,  3.71428571],
       [ 1.85714286,  6.5       ,  3.71428571,  0.92857143]])

In [83]:
A

array([[ 2, 12,  6, 18],
       [ 7, 42, 21, 63],
       [ 4, 24, 12, 36],
       [ 1,  6,  3,  9]])

In [ ]:
def als(M, d, lam, iters):
    #intialize U
    #shape should be rank x M[rows]
    U = np.random.random((d, M.shape[0]))
    
    #storage for errors per iteration
    train_errors = []

    

    
    #iterate
    for _ in range(iters):
        
        #hold U constant and update V
        V = np.linalg.pinv(U.dot(U.T) + lam * np.eye(d)).dot(U.dot(M))
 
        #hold V constant and update U
        U = np.linalg.pinv(V.dot(V.T) + lam * np.eye(d)).dot(V.dot(M.T))

    return U,V